In [13]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

## Umbralizado
Primero umbralizamos la imagen con un umbralizado básico y luego con otro adaptativo y luego obtenemos los contornos. De momento el normal da mejores resultados

In [79]:
# Umbralizado normal
imagen = cv2.imread("test_ocr_panels/00006_0.png", 0)
imagenUmbralizada = imagen.copy()
imagenUmbralizada[imagen>80] = 255
imagenUmbralizada[imagen<=80] = 0
contornos, _ = cv2.findContours(imagenUmbralizada, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
imagenContornos = np.zeros_like(imagen)
cv2.drawContours(imagenContornos, contornos, -1, (255, 255, 255), 1)
cv2.imshow("Imagen Umbralizada", imagenUmbralizada)
cv2.imshow("Imagen con Contornos", imagenContornos)
cv2.waitKey(0)
cv2.destroyAllWindows()
#plt.imshow(imagenContornos, cmap="gray")

In [71]:
# Umbralizado adaptativo
imagen = cv2.imread("test_ocr_panels/00006_0.png", 0)
imagenUmbralizada = cv2.adaptiveThreshold(imagen,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,3)
contornos, _ = cv2.findContours(imagenUmbralizada, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
imagenContornos = np.zeros_like(imagen)
cv2.drawContours(imagenContornos, contornos, -1, (255, 255, 255), 1)
cv2.imshow("Imagen Umbralizada", imagenUmbralizada)
cv2.imshow("Imagen con Contornos", imagenContornos)
cv2.waitKey(0)
cv2.destroyAllWindows()
#plt.imshow(imagenContornos, cmap="gray")

## MSER
Acto seguido tendremos que detectar los caracteres del panel usando MSER

In [81]:
# MSER
def mser(imagenUmbralizada, imagen):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    imagenUmbralizada = cv2.dilate(imagenUmbralizada, kernel)
    mser = cv2.MSER_create(delta=5, max_variation=0.2, min_area=50, max_area=900)
    polygons, _ = mser.detectRegions(imagenUmbralizada)
    copy = imagen.copy()
    rectangles = []
    for p in polygons:
        x, y, w, h = cv2.boundingRect(p)
        if (h / w > 1 and h / w <= 2):
            cv2.rectangle(copy, (x, y), (x + w, y + h), (0, 0, 255), 1)
            rectangles.append((x,y,w,h))
    cv2.imshow("Regiones detectadas por MSER", copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return rectangles

imagen = cv2.imread("test_ocr_panels/00006_0.png")
rectanglesDetected = mser(imagenContornos, imagen)

## Agrupación de los centros
Una vez tenemos los rectángulos de cada caracter, tendremos que calcular sus centros para luego pasarselo al RANSAC. Antes tendremos que agrupar estos centros en conjuntos de palabras en función de su proximidad con cada uno de los puntos en el eje X

In [85]:
from sklearn import linear_model
from sklearn.cluster import DBSCAN

def agruparCentros(rectanglesDetected, imagen):
    # Dibujamos los centros de los caracteres
    copy = imagen.copy()
    centros = []
    for rectangle in rectanglesDetected:
        x, y, w, h = rectangle
        center = (x + (w//2), y + (h//2))
        cv2.circle(copy, (center[0], center[1]), radius=1, color=(0, 0, 255), thickness=2)
        centros.append(list(center))
    # Utilizamos DBSCAN para agrupar los centros en grupos
    dbscan = DBSCAN(eps=35, min_samples=1)
    y_pred = dbscan.fit_predict(centros)
    # Construimos los conjuntos a partir de las etiquetas obtenidas
    setCenters = []
    for _ in range(len(np.unique(y_pred))):
        setCenters.append([])
    for i, center in enumerate(centros):
        index = y_pred[i]
        if (len(setCenters[index]) == 0):
            setCenters[index] = [center]
        else:
            elem = setCenters[index]
            elem.append(center)
            setCenters[index] = elem
    # Aplicamos RANSAC para cada uno de los conjuntos
    ransac = linear_model.RANSACRegressor()
    for set in setCenters:
        if (len(set) > 1):
            centrosX = [sublista[0] for sublista in set]
            centrosY = [sublista[1] for sublista in set]
            centrosX = np.array(centrosX).reshape(-1, 1)
            centrosY = np.array(centrosY)

            ransac.fit(centrosX, centrosY)
            x0 = centrosX.min()
            x1 = centrosX.max()  # Usamos el rango de nuestros datos
            y0 = ransac.estimator_.intercept_
            y1 = x1 * ransac.estimator_.coef_[0] + ransac.estimator_.intercept_
            start_point = (int(x0), int(y0))
            end_point = (int(x1), int(y1))
            cv2.line(copy, start_point, end_point, color=(0, 255, 0), thickness=1)
    cv2.imshow("Regiones detectadas por MSER", copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

agruparCentros(rectanglesDetected, imagen)

In [83]:
from sklearn.linear_model import RANSACRegressor

def ransac(rectanglesDetected, imagen):
    ransac = RANSACRegressor()
    copy = imagen.copy()
    centrosX = []
    centrosY = []
    for rectangle in rectanglesDetected:
        x, y, w, h = rectangle
        center = (x + (w//2), y + (h//2))
        cv2.circle(copy, (center[0], center[1]), radius=1, color=(255, 255, 255), thickness=2)
        centrosX.append(center[0])
        centrosY.append(center[1])
    centrosX = np.array(centrosX).reshape(-1, 1)
    centrosY = np.array(centrosY)

    ransac.fit(centrosX, centrosY)
    x0 = centrosX.min()
    x1 = centrosX.max()  # Usamos el rango de nuestros datos
    y0 = ransac.estimator_.intercept_
    y1 = x1 * ransac.estimator_.coef_[0] + ransac.estimator_.intercept_
    start_point = (int(x0), int(y0))
    end_point = (int(x1), int(y1))

    # Dibujar la línea en la imagen
    #cv2.line(copy, start_point, end_point, color=(0, 0, 255), thickness=2)

    
    cv2.imshow("Regiones detectadas por MSER", copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

ransac(rectanglesDetected, imagen)